In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/RECSData/recs2009_public.csv")

data

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/RECSData/recs2009_public.csv'

## Dropping redundant cols

In [ ]:
# Select only numerical columns
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
cat_cols = data.select_dtypes(include=['object']).columns

num_data= data[numerical_cols]
cat_data= data[cat_cols]
num_data.shape,cat_data.shape

NameError: name 'data' is not defined

In [ ]:
num_data.head()

,DOEID,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,...,PERIODKR,SCALEKER,HDD50,CDD80,GND_HDD65,WSF,OA_LAT,GWT,DesignDBT99,DesignDBT1
0,1,2,4,12,2,2471.68,4742,1080,4953,1271,...,-2,-2,2117,56,4250,0.48,6,56,9,96
1,2,4,10,26,2,8599.17,2662,199,2688,143,...,-2,-2,62,26,2393,0.61,0,64,38,73
2,3,1,1,1,5,8969.92,6233,505,5741,829,...,-2,-2,2346,49,5654,0.48,3,52,12,88
3,4,2,3,7,2,18003.64,6034,672,5781,868,...,-2,-2,2746,0,4941,0.55,4,55,7,87
4,5,1,1,1,3,5999.61,5388,702,5313,797,...,-2,-2,2251,0,5426,0.61,4,50,13,90


In [ ]:
columns_to_check = num_data.columns.tolist()
# counter of index in columns_to_check
# drop some cols, stored in cols_to_drop

Remove columns that are highly correlated to each other

In [ ]:

# Make a copy of the DataFrame columns
columns_to_check = num_data.columns.tolist()
cols_dropped=[]
# Iterate through each column in the DataFrame
i=0
while i< len(columns_to_check):
    col_x=columns_to_check[i]

    # Create a list to store columns to drop
    cols_to_drop = []

    # Iterate through each column (except col_x)
    for col_y in columns_to_check:
        if col_y != col_x:
            # Calculate correlation between col_x and col_y
            corr = num_data[col_x].corr(num_data[col_y])

            # If correlation is higher than 0.95, add col_y to cols_to_drop
            if corr > 0.95:
                cols_to_drop.append(col_y)

    # Drop highly correlated columns from the DataFrame
    if len(cols_to_drop) > 0:
        # print(f"Removing columns highly correlated with {col_x}: {cols_to_drop}")
        num_data.drop(columns=cols_to_drop, inplace=True)
        # Update columns_to_check after dropping highly correlated columns
        columns_to_check = [col for col in columns_to_check if col not in cols_to_drop]
        cols_dropped.extend(cols_to_drop)

    if (i%15 == 0):
        print(i/len(columns_to_check))
    i+=1



In [ ]:
num_data.shape

(12083, 781)

In [ ]:
len(cols_dropped)

154

Dropping columns that aren't highly correlated with target

In [ ]:
col_target = 'KWH'

# Calculate correlation of each feature column with the target column
correlation_with_target = num_data.corr()[col_target].abs()

# Drop columns whose absolute correlation with the target column is less than 10%
columns_to_drop = correlation_with_target[correlation_with_target < 0.1].index

# Drop the columns from the DataFrame
num_data.drop(columns=columns_to_drop, inplace=True)


num_data.shape

(12083, 284)

In [ ]:
num_data

,TYPEHUQ,HDD65,CDD65,Climate_Region_Pub,AIA_Zone,KOWNRENT,CONDCOOP,YEARMADE,CONVERSION,NUMFLRS,...,TOTALBTUWTH,TOTALBTUOTH,TOTALDOL,TOTALDOLWTH,KAVALNG,CDD80,WSF,OA_LAT,GWT,DesignDBT1
0,2,4742,1080,4,3,1,-2,2004,-2,-2,...,10128,26363,1315,211,-2,56,0.48,6,56,96
1,2,2662,199,5,4,2,-2,1998,-2,-2,...,30110,29653,1293,287,1,26,0.61,0,64,73
2,5,6233,505,1,2,2,-2,1965,-2,2,...,7933,6781,1327,115,3,49,0.48,3,52,88
3,2,6034,672,1,2,1,-2,1985,-2,-2,...,6440,20438,1398,179,3,0,0.55,4,55,87
4,3,5388,702,1,3,1,1,1983,-2,-2,...,13884,8607,1558,237,1,0,0.61,4,50,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12078,2,7671,332,1,1,1,-2,1985,-2,-2,...,6471,17356,1541,253,3,39,0.59,0,48,84
12079,4,2085,2844,2,5,2,-2,1980,1,-2,...,3645,3262,543,54,1,393,0.50,0,56,101
12080,2,7935,472,1,1,1,-2,1993,-2,-2,...,24391,42930,2093,204,1,22,0.58,1,46,89
12081,2,5834,770,1,2,1,-2,1930,-2,-2,...,12416,12960,1023,108,1,0,0.54,1,53,93


## Now convert all non-target attributes to their full names, so that we can understand what they stand for easily

In [ ]:
layout=pd.read_csv("/content/drive/MyDrive/RECSData/public_layout.csv")


In [ ]:
mapping = dict(zip(layout['Variable Name'], layout['Variable Label']))
cat_data = cat_data.rename(columns=mapping)
num_data = num_data.rename(columns=mapping)
data= data.rename(columns=mapping)

Analysis on numerical columns only

In [ ]:
cols=list(num_data.columns)

In [ ]:
data= data[cols]

In [ ]:
data = data.loc[:, ~data.columns.duplicated()]
data.head() #dropping any duplicate cols

,Type of housing unit,"Heating degree days in 2009, base temperature 65F","Cooling degree days in 2009, base temperature 65F",Building America Climate Region (collapsed for public file),"AIA Climate Zone, based on average temperatures from 1981 - 2010","Housing unit is owned, rented, or occupied without payment of rent",Housing unit part of condominium or cooperative,Year housing unit was built,Converted 2-4 unit apartment building,Number of floors in a 5+ unit apartment building,...,"Total usage for water heating, in thousand BTU, 2009","Total usage for appliances, electronics, lighting, and miscellaneous uses, in thousand BTU, 2009","Total cost, in whole dollars, 2009","Total cost for water heating, in whole dollars, 2009",Natural gas end uses included in Energy Supplier Survey billing data,"Cooling degree days in 2009, base temperature 80F",Weather & shielding factor for the housing unit,Annual value for latent heat infiltration from outside air into the housing unit,Annual average ground water temperature (F) for 2009,Dry bulb design temperature (F) expected to be exceeded 1% of the time
0,2,4742,1080,4,3,1,-2,2004,-2,-2,...,10128,26363,1315,211,-2,56,0.48,6,56,96
1,2,2662,199,5,4,2,-2,1998,-2,-2,...,30110,29653,1293,287,1,26,0.61,0,64,73
2,5,6233,505,1,2,2,-2,1965,-2,2,...,7933,6781,1327,115,3,49,0.48,3,52,88
3,2,6034,672,1,2,1,-2,1985,-2,-2,...,6440,20438,1398,179,3,0,0.55,4,55,87
4,3,5388,702,1,3,1,1,1983,-2,-2,...,13884,8607,1558,237,1,0,0.61,4,50,90


acquiring only those columns without too many missing values

In [ ]:
num_cols = data.select_dtypes(include=['float64', 'int64']).columns
# dis_cols=[]
len(num_cols)

284

In [ ]:
nonsense= [str(col) for col in num_cols if (data[col]==-2).mean()>=0.1]     #we're checking if there's more than 10% -2's in a col, if yes, drop
#-2 is a value we see very often. there's a very high chance it means missing value because in columns age group of 5th, 6th, 7th, 8th member of the family >97% of values were -2.
len(nonsense)

130

In [ ]:
data.drop(columns=nonsense, inplace=True)
data.head()

,Type of housing unit,"Heating degree days in 2009, base temperature 65F","Cooling degree days in 2009, base temperature 65F",Building America Climate Region (collapsed for public file),"AIA Climate Zone, based on average temperatures from 1981 - 2010","Housing unit is owned, rented, or occupied without payment of rent",Year housing unit was built,Number of bedrooms,Number of full bathrooms,Number of half bathrooms,...,"Total usage, in thousand BTU, 2009","Total usage for water heating, in thousand BTU, 2009","Total usage for appliances, electronics, lighting, and miscellaneous uses, in thousand BTU, 2009","Total cost, in whole dollars, 2009","Total cost for water heating, in whole dollars, 2009","Cooling degree days in 2009, base temperature 80F",Weather & shielding factor for the housing unit,Annual value for latent heat infiltration from outside air into the housing unit,Annual average ground water temperature (F) for 2009,Dry bulb design temperature (F) expected to be exceeded 1% of the time
0,2,4742,1080,4,3,1,2004,4,1,2,...,63006,10128,26363,1315,211,56,0.48,6,56,96
1,2,2662,199,5,4,2,1998,3,2,1,...,103460,30110,29653,1293,287,26,0.61,0,64,73
2,5,6233,505,1,2,2,1965,1,1,0,...,58716,7933,6781,1327,115,49,0.48,3,52,88
3,2,6034,672,1,2,1,1985,3,2,1,...,76401,6440,20438,1398,179,0,0.55,4,55,87
4,3,5388,702,1,3,1,1983,2,1,1,...,59809,13884,8607,1558,237,0,0.61,4,50,90


In [ ]:
# for i in range(len(num_cols)):
#   try:
#     if (data[num_cols[i]].nunique()) <= 30:
#       dis_cols.append(num_cols[i])
#   except:
#     print(num_cols[i])
# len(dis_cols)

In [ ]:
TARGET= "Total Site Electricity usage, in kilowatt-hours, 2009"

In [ ]:
#dis_cols

In [ ]:
#len(dis_cols)

for every discrete attribute:

step 0:

get a code, to find % of -2's in every column

step 1:
check range of the attribute
bar chart of the attributes, so if we have say, an excess of -2's, easily visible

if percentage of -2's > 50%, drop


step 2:
if similar cols exist, drop one


step 3
if less than 50%
check individual corr score w.r.t target
if abs(Score) <0.2, drop

else.............................
find some imputation

In [ ]:
#dis_data=data[dis_cols]

In [ ]:
#dis_data.head()

In [ ]:
# cont_data=data.drop(columns=dis_cols,inplace=True)
# cont_data.head()

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

the below cell says some column names are not found only

In [ ]:
# num_sense_cols = [item for item in num_cols if item not in nonsense]
# num_data=data[num_sense_cols]

categorical columns feature selection

In [ ]:
cat_data.dtypes

Housing unit in Census Metropolitan Statistical Area or Micropolitan Statistical Area     object
Housing unit classified as urban or rural by Census                                       object
Number of kerosene 'cash and carry' purchases                                             object
Gallons per kerosene 'cash and carry' purchase                                            object
International Energy Conservation Code (IECC) climate zone (collapsed for public file)    object
dtype: object

In [ ]:
cat_data["Gallons per kerosene 'cash and carry' purchase"].unique()

array([-2, 1, 5, 10, '-2', '10', '15', '1', '8', '.', '5', '3', '2', '77',
       30, 3, 15, 8, 55, 2, 20], dtype=object)

In [ ]:
cat_data.drop(columns=[cat_data.columns[2:4]],inplace=True)
cat_data

KeyError: "[('International Energy Conservation Code (IECC) climate zone (collapsed for public file)',)] not found in axis"

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
cat_encoded = cat_data.apply(LabelEncoder().fit_transform)

In [ ]:
cat_encoded, cat_encoded.dtypes

(       Housing unit in Census Metropolitan Statistical Area or Micropolitan Statistical Area  \
 0                                                      0                                       
 1                                                      0                                       
 3                                                      0                                       
 4                                                      0                                       
 5                                                      0                                       
 ...                                                  ...                                       
 12078                                                  0                                       
 12079                                                  0                                       
 12080                                                  0                                       
 12081                        

In [ ]:
data = pd.concat([data,cat_encoded], axis=1)
data.head()

,Type of housing unit,"Heating degree days in 2009, base temperature 65F","Cooling degree days in 2009, base temperature 65F",Building America Climate Region (collapsed for public file),"AIA Climate Zone, based on average temperatures from 1981 - 2010","Housing unit is owned, rented, or occupied without payment of rent",Year housing unit was built,Number of bedrooms,Number of full bathrooms,Number of half bathrooms,...,"Total cost, in whole dollars, 2009","Total cost for water heating, in whole dollars, 2009","Cooling degree days in 2009, base temperature 80F",Weather & shielding factor for the housing unit,Annual value for latent heat infiltration from outside air into the housing unit,Annual average ground water temperature (F) for 2009,Dry bulb design temperature (F) expected to be exceeded 1% of the time,Housing unit in Census Metropolitan Statistical Area or Micropolitan Statistical Area,Housing unit classified as urban or rural by Census,International Energy Conservation Code (IECC) climate zone (collapsed for public file)
0,2,4742,1080,4,3,1,2004,4,1,2,...,1315,211,56,0.48,6,56,96,0.0,1.0,5.0
1,2,2662,199,5,4,2,1998,3,2,1,...,1293,287,26,0.61,0,64,73,0.0,1.0,4.0
2,5,6233,505,1,2,2,1965,1,1,0,...,1327,115,49,0.48,3,52,88,NaN,NaN,NaN
3,2,6034,672,1,2,1,1985,3,2,1,...,1398,179,0,0.55,4,55,87,0.0,1.0,7.0
4,3,5388,702,1,3,1,1983,2,1,1,...,1558,237,0,0.61,4,50,90,0.0,1.0,7.0


In [ ]:
data.to_csv("/content/drive/MyDrive/RECSData/recs2009_public_trimmed.csv",index=False)